In [31]:
import io
import json
import os
import sys
import pandas as pd
from bs4 import BeautifulSoup as bs

import requests
from requests.sessions import Session

import numpy as np
from scipy import stats
import datetime
import time
import statistics as st
import concurrent
from threading import Thread,local
import asyncio
import aiohttp
import tqdm

from sklearn.linear_model import LinearRegression

from collections.abc import MutableMapping

In [32]:
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}

hippo_letrot = ["LE MONT-SAINT-MICHEL-PONTORSON", "BORDEAUX", "LE CROISE-LAROCHE","CAGNES-SUR-MER","MESLAY-DU-MAINE"]
hippo_pmu =  ["LE MONT SAINT MICHEL", "LE BOUSCAT", "LE CROISE LAROCHE","CAGNES/MER","MESLAY DU MAINE"]

In [33]:
import nest_asyncio
nest_asyncio.apply()

thread_local = local()

# def get_session() -> Session:
#     if not hasattr(thread_local, 'session'):
#         thread_local.session =  requests.Session()
#     return thread_local.session

# def get_request_with_session(url:str):
#     session = get_session()
#     with session.get(url, headers=headers) as response:
#         return response

def gen_rows(df):
    for row in df.itertuples(index=False):
        yield row._asdict()

### OOP

In [34]:
class Programme():
    def __init__(self, debut, fin):
        self.date_debut = datetime.date.fromisoformat("-".join(debut.split("-")[::-1]))
        self.date_fin = datetime.date.fromisoformat("-".join(fin.split("-")[::-1]))
        
        n_days = self.date_fin - self.date_debut
        cur = max(self.date_fin - datetime.timedelta(days=90), self.date_debut)
        
        intervalle_date = [self.date_fin, cur]
        
        while cur > self.date_debut:
            cur = max(self.date_debut, cur - datetime.timedelta(days=90))
            
            intervalle_date.append(cur)        
        self.intervalles = [(i,j + datetime.timedelta(days=1)) for i,j in zip(intervalle_date, intervalle_date[1:])]
        
        
        loop = asyncio.get_event_loop()
        programme = loop.run_until_complete(asyncio.gather(*[self._get_programme_from_letrot(inter) for inter in self.intervalles]))
        
        programme = [item for sublist in programme for item in sublist]
        
        self.programme = pd.DataFrame(programme)

    async def combined_prog(self):
        return await asyncio.gather(*[self._get_programme_from_letrot(inter) for inter in self.intervalles])


    async def _get_pmu_program(self, session, date):
        date_pmu = date.strftime("%d%m%Y")
        async with session.get(f"https://online.turfinfo.api.pmu.fr/rest/client/65/programme/{date_pmu}/") as res:
            try:
                return await res.json()
            except:
                return None

    async def _get_programme_from_letrot(self, date):
        debut = date[1].strftime("%d-%m-%Y")
        fin = date[0].strftime("%d-%m-%Y")
#         print(debut)
        programme = []
        
#         async with aiohttp.ClientSession() as session:
#             async with session.get(f"https://www.letrot.com/fr/courses/calendrier-resultats?publish_up={debut}&publish_down={fin}", headers=headers) as response:
#                  r = await response.text()
        
        
        num_days = (date[0] - date[1]).days + 1
        date_list = [date[0] - datetime.timedelta(days=x) for x in range(num_days)]
        prog_pmu = {}
        
        tasks = []
        async with aiohttp.ClientSession() as session:
            async with session.get(f"https://www.letrot.com/fr/courses/calendrier-resultats?publish_up={debut}&publish_down={fin}", headers=headers) as response:
                 r = await response.text()
            for d in date_list:
                tasks.append(asyncio.ensure_future(self._get_pmu_program(session, d)))
            res_prog_pmu = await asyncio.gather(*tasks)
        soup = bs(r, "html.parser")
        reunion_raw = soup.find_all("a", {"class": "racesHippodrome"})
        current_date_reunion = "0"
        current_programme = {}    
            
            
            
        for i in range(len(res_prog_pmu)):
            prog_pmu.update({date_list[i].strftime("%Y-%m-%d"): res_prog_pmu[i]})

        
        
        for i in range(len(reunion_raw)):
            reunion = reunion_raw[i]
            date = reunion.get("href").split("/")[-2]
            hippodrome = reunion.text[2:].strip()
            for i in range(len(hippo_letrot)):
                hippodrome = hippodrome.replace(hippo_letrot[i], hippo_pmu[i])
            
            hippodrome = hippodrome.replace(" (A ", " ").replace(")", "")
            date_pmu = "".join(date.split("-")[::-1])
            
            if date in prog_pmu:
                current_programme = prog_pmu[date]
            else:
                continue
                
            numReunion = 0
            try:
                for reunion_pmu in current_programme["programme"]["reunions"]:
                    if hippodrome in reunion_pmu["hippodrome"]["libelleCourt"]:
                        numReunion = reunion_pmu["numOfficiel"]
            except:
                continue
            
            if numReunion == 0:
                continue
            course = {"date": date, "idHippo": reunion.get("href").split("/")[-1], "Hippodrome": hippodrome, "lien": reunion.get("href")}
            course["numReunion"] = numReunion
            programme.append(course)
        return programme

In [36]:
tps = time.time()
t = Programme("01-01-2019", "31-12-2020")
print("Fini en ", time.time() - tps, "s")
t.programme

Fini en  96.6271390914917 s


,date,idHippo,Hippodrome,lien,numReunion
0,2020-10-03,1400,CAEN,/stats/courses/programme/2020-10-03/1400,3
1,2020-10-03,8002,AMIENS,/stats/courses/programme/2020-10-03/8002,10
2,2020-10-03,0303,MOULINS,/stats/courses/programme/2020-10-03/0303,8
3,2020-10-05,5102,REIMS,/stats/courses/programme/2020-10-05/5102,1
4,2020-10-05,5901,LE CROISE LAROCHE,/stats/courses/programme/2020-10-05/5901,7
...,...,...,...,...,...
707,2019-01-03,7500,VINCENNES,/stats/courses/programme/2019-01-03/7500,1
708,2019-01-06,7500,VINCENNES,/stats/courses/programme/2019-01-06/7500,1
709,2019-01-07,7500,VINCENNES,/stats/courses/programme/2019-01-07/7500,1
710,2019-01-08,7500,VINCENNES,/stats/courses/programme/2019-01-08/7500,1


In [37]:
tps = time.time()
t2 = Programme("01-01-2019", "31-12-2020")
print("Fini en ", time.time() - tps, "s")
t2.programme

Fini en  97.97451639175415 s


,date,idHippo,Hippodrome,lien,numReunion
0,2020-10-06,8301,HYERES,/stats/courses/programme/2020-10-06/8301,3
1,2020-10-06,7500,VINCENNES,/stats/courses/programme/2020-10-06/7500,4
2,2020-10-07,7502,ENGHIEN,/stats/courses/programme/2020-10-07/7502,1
3,2020-10-08,7502,ENGHIEN,/stats/courses/programme/2020-10-08/7502,4
4,2020-10-13,7500,VINCENNES,/stats/courses/programme/2020-10-13/7500,4
...,...,...,...,...,...
455,2019-01-03,7500,VINCENNES,/stats/courses/programme/2019-01-03/7500,1
456,2019-01-04,0601,CAGNES/MER,/stats/courses/programme/2019-01-04/0601,3
457,2019-01-06,7500,VINCENNES,/stats/courses/programme/2019-01-06/7500,1
458,2019-01-08,7500,VINCENNES,/stats/courses/programme/2019-01-08/7500,1


In [41]:
tps = time.time()
t3 = Programme("01-01-2019", "31-12-2020")
print("Fini en ", time.time() - tps, "s")
t3.programme

Fini en  97.79292869567871 s


,date,idHippo,Hippodrome,lien,numReunion
0,2020-10-06,8301,HYERES,/stats/courses/programme/2020-10-06/8301,3
1,2020-10-06,7500,VINCENNES,/stats/courses/programme/2020-10-06/7500,4
2,2020-10-07,7502,ENGHIEN,/stats/courses/programme/2020-10-07/7502,1
3,2020-10-08,7502,ENGHIEN,/stats/courses/programme/2020-10-08/7502,4
4,2020-10-09,4201,FEURS,/stats/courses/programme/2020-10-09/4201,3
...,...,...,...,...,...
606,2019-01-05,7500,VINCENNES,/stats/courses/programme/2019-01-05/7500,1
607,2019-01-06,7500,VINCENNES,/stats/courses/programme/2019-01-06/7500,1
608,2019-01-08,7500,VINCENNES,/stats/courses/programme/2019-01-08/7500,1
609,2019-01-09,0601,CAGNES/MER,/stats/courses/programme/2019-01-09/0601,1


In [5]:
P = Programme("01-01-2022", "13-12-2022")
P.programme.head()

,date,idHippo,Hippodrome,lien,numReunion
0,2022-09-20,7500,VINCENNES,/stats/courses/programme/2022-09-20/7500,4
1,2022-10-04,7500,VINCENNES,/stats/courses/programme/2022-10-04/7500,4
2,2022-10-07,8301,HYERES,/stats/courses/programme/2022-10-07/8301,3
3,2022-10-07,7500,VINCENNES,/stats/courses/programme/2022-10-07/7500,1
4,2022-10-14,7500,VINCENNES,/stats/courses/programme/2022-10-14/7500,1


In [6]:
P2 = Programme("01-01-2021", "30-11-2021")
P2.programme

,date,idHippo,Hippodrome,lien,numReunion
0,2021-09-03,5305,LAVAL,/stats/courses/programme/2021-09-03/5305,5
1,2021-09-03,6701,STRASBOURG,/stats/courses/programme/2021-09-03/6701,3
2,2021-09-03,7500,VINCENNES,/stats/courses/programme/2021-09-03/7500,1
3,2021-09-03,0202,LA CAPELLE,/stats/courses/programme/2021-09-03/0202,6
4,2021-09-04,7500,VINCENNES,/stats/courses/programme/2021-09-04/7500,1
...,...,...,...,...,...
630,2021-03-02,4417,PONTCHATEAU,/stats/courses/programme/2021-03-02/4417,3
631,2021-03-03,7616,MAUQUENCHY,/stats/courses/programme/2021-03-03/7616,5
632,2021-03-03,7500,VINCENNES,/stats/courses/programme/2021-03-03/7500,1
633,2021-03-04,7500,VINCENNES,/stats/courses/programme/2021-03-04/7500,1


In [42]:
P3 = pd.concat([t.programme,t2.programme,t3.programme], axis=0).reset_index(drop=True).drop_duplicates()

In [46]:
P3.drop_duplicates(inplace=True)
P3.reset_index(inplace=True,drop=True)
P3

,date,idHippo,Hippodrome,lien,numReunion
0,2020-10-03,1400,CAEN,/stats/courses/programme/2020-10-03/1400,3
1,2020-10-03,8002,AMIENS,/stats/courses/programme/2020-10-03/8002,10
2,2020-10-03,0303,MOULINS,/stats/courses/programme/2020-10-03/0303,8
3,2020-10-05,5102,REIMS,/stats/courses/programme/2020-10-05/5102,1
4,2020-10-05,5901,LE CROISE LAROCHE,/stats/courses/programme/2020-10-05/5901,7
...,...,...,...,...,...
884,2019-03-18,1400,CAEN,/stats/courses/programme/2019-03-18/1400,2
885,2019-04-05,7500,VINCENNES,/stats/courses/programme/2019-04-05/7500,1
886,2019-04-05,5305,LAVAL,/stats/courses/programme/2019-04-05/5305,7
887,2019-01-05,7500,VINCENNES,/stats/courses/programme/2019-01-05/7500,1


In [47]:
class Courses():
    def __init__(self, programme) -> None:
        if isinstance(programme, Programme):
            self.programme = programme.programme
        elif isinstance(programme, pd.DataFrame):
            self.programme = programme
        self.courses = self._get_all_course_in_programme()

    def _get_all_course_in_programme(self):
        courses = []  

        def _request_race(row):
            courses_list = []
            try:
                url = f"https://www.letrot.com/{row['lien']}/json"
                date_pmu = "".join(row["date"].split("-")[::-1])    
                r = requests.get(url, headers=headers)
                courses = r.json()
                for c in courses["course"]:
                    if c["discipline"] == "Attelé":
                        course_id = row["date"].replace("-", "") + str(row["idHippo"]) + str(c["numCourse"])
                        courses_list.append({"date": row["date"], "id": course_id, "numReunion": row["numReunion"], "hippodrome": courses["nomHippodrome"], "idHippo": row["idHippo"],**c})
                return courses_list
            except:
                pass
            
        def gen_rows(df):
            for row in df.itertuples(index=False):
                yield row._asdict()

        with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
            res = executor.map(_request_race, gen_rows(self.programme))
            
            for i in res:
                courses.extend(i)
            
        return pd.DataFrame(courses)

In [49]:
courses = Courses(P3)

In [50]:
courses.courses

,date,id,numReunion,hippodrome,idHippo,heureCourse,discipline,numCourse,prix,allocation,distance,categorie,typePiste,conditionsEngagement,hasTracking,hasVideoHeat,statut,classement,linkPrix,replay
0,2020-10-03,2020100314003,3,CAEN,1400,13:19,Attelé,3,PRIX DE FORMENTIN,21 000,2 450,Course C,,Course Nationale<br />LeTROT Open des Régions ...,False,False,16,8 - 2 - 11 - 9 - 1,https://www.letrot.com/stats/fiche-course/2020...,"<a href=""https://www.letrot.com/fr/replay-cour..."
1,2020-10-03,2020100314004,3,CAEN,1400,13:47,Attelé,4,PRIX DE FOLLIGNY,16 000,2 450,Course E,,"Pour poulains entiers et hongres de 4 ans, n'a...",False,False,16,13 - 14 - 16 - 3 - 18,https://www.letrot.com/stats/fiche-course/2020...,"<a href=""https://www.letrot.com/fr/replay-cour..."
2,2020-10-03,2020100314005,3,CAEN,1400,14:20,Attelé,5,PRIX DE BOISSEY,18 000,2 450,Course E,,Course Nationale<br />LeTROT Open des Régions ...,False,False,16,16 - 18 - 11 - 3 - 4,https://www.letrot.com/stats/fiche-course/2020...,"<a href=""https://www.letrot.com/fr/replay-cour..."
3,2020-10-03,2020100314007,3,CAEN,1400,15:32,Attelé,7,PRIX DE LONGVILLERS,20 000,2 450,Course D,,"Pour 6 ans, n'ayant pas gagné 147.000. - Recul...",False,False,16,2 - 13 - 7 - 8 - 12,https://www.letrot.com/stats/fiche-course/2020...,"<a href=""https://www.letrot.com/fr/replay-cour..."
4,2020-10-03,2020100314008,3,CAEN,1400,16:07,Attelé,8,PRIX D'AUTOMNE,31 000,2 450,Course C,,"Pour 6 à 9 ans inclus, n'ayant pas gagné 400.0...",False,False,16,5 - 17 - 9 - 2 - 15,https://www.letrot.com/stats/fiche-course/2020...,"<a href=""https://www.letrot.com/fr/replay-cour..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5725,2019-01-11,2019011175001,1,VINCENNES,7500,13:45,Attelé,1,PRIX DE MAURIAC,56 000,2 850,Course A,,"Course Européenne<br />Pour 5 ans (E), ayant g...",True,False,16,9 - 6 - 10 - 3 - 18,https://www.letrot.com/stats/fiche-course/2019...,"<a href=""https://www.letrot.com/fr/replay-cour..."
5726,2019-01-11,2019011175003,1,VINCENNES,7500,14:55,Attelé,3,PRIX D'EVIAN,47 000,2 700,Course B,,"Pour pouliches de 4 ans (F), n'ayant pas gagné...",True,False,16,15 - 5 - 8 - 2 - 14,https://www.letrot.com/stats/fiche-course/2019...,"<a href=""https://www.letrot.com/fr/replay-cour..."
5727,2019-01-11,2019011175004,1,VINCENNES,7500,15:31,Attelé,4,PRIX DE RANES,45 000,2 700,Course C,,"Course Européenne<br />Pour juments de 6, 7 et...",True,False,16,4 - 7 - 2 - 1 - 8,https://www.letrot.com/stats/fiche-course/2019...,"<a href=""https://www.letrot.com/fr/replay-cour..."
5728,2019-01-11,2019011175006,1,VINCENNES,7500,16:40,Attelé,6,PRIX DE VIC-BIGORRE,33 000,2 100,Course C,,Départ à l'autostart<br />Pour pouliches de 3 ...,False,False,16,2 - 4 - 3 - 7 - 15,https://www.letrot.com/stats/fiche-course/2019...,"<a href=""https://www.letrot.com/fr/replay-cour..."


In [51]:
courses.courses.to_csv("data/courses1920.csv")

In [5]:
c = pd.read_csv("data/courses20212022.csv", index_col=0)
c

,date,id,numReunion,hippodrome,idHippo,heureCourse,discipline,numCourse,prix,allocation,distance,categorie,typePiste,conditionsEngagement,hasTracking,hasVideoHeat,statut,classement,linkPrix,replay
0,2022-09-15,2022091576162,4,MAUQUENCHY,7616,14:30,Attelé,2,PRIX LE VINCENNES DE NEUVILLE LES DIEPPE,20 000,2 850,Course E,NaN,Course Nationale<br />LeTROT Open des Régions ...,False,True,16,16 - 15 - 14 - 6 - 11,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
1,2022-09-15,2022091576164,4,MAUQUENCHY,7616,15:35,Attelé,4,PRIX LE BALTO BAR DE GOURNAY EN BRAY,23 000,2 850,Course E,NaN,Course Nationale<br />LeTROT Open des Régions ...,False,False,16,9 - 17 - 15 - 10 - 3,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
2,2022-09-15,2022091576165,4,MAUQUENCHY,7616,16:10,Attelé,5,PRIX LE MAG'PRESSE DE BOISGUILLAUME,20 000,2 150,Course E,NaN,Course Nationale<br />LeTROT Open des Régions ...,False,True,16,6 - 8 - 4 - 11 - 5,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
3,2022-09-15,2022091576166,4,MAUQUENCHY,7616,16:45,Attelé,6,PRIX LE BAR DES 4 RIVIERES D'ARGUEIL,20 000,2 150,Course E,NaN,Apprentis-lads-jockeys<br />Départ à l'autosta...,False,False,16,6 - 5 - 11 - 13 - 12,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
4,2022-09-15,2022091576167,4,MAUQUENCHY,7616,17:20,Attelé,7,PRIX AUX GARENNES DE GARENNES SUR EURE,6 000,2 150,Course G,NaN,CHALLENGE UNAT<br />Amateurs<br />Départ à l'a...,False,False,16,3 - 14 - 1 - 13 - 4,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9064,2021-03-23,2021032375022,4,ENGHIEN,7502,16:06,Attelé,2,PRIX DU PONT SULLY,29 000,2 150,Course E,NaN,Départ à l'autostart<br />Pour poulains entier...,True,False,16,7 - 3 - 11 - 10 - 9,https://www.letrot.com/stats/fiche-course/2021...,"<a href=""https://www.letrot.com/fr/replay-cour..."
9065,2021-03-23,2021032375024,4,ENGHIEN,7502,17:05,Attelé,4,PRIX DES COTES D'ARMOR,33 000,2 875,Course D,NaN,"Pour pouliches de 4 ans, n'ayant pas gagné 40....",True,False,16,3 - 6 - 4 - 10 - 9,https://www.letrot.com/stats/fiche-course/2021...,"<a href=""https://www.letrot.com/fr/replay-cour..."
9066,2021-03-23,2021032375025,4,ENGHIEN,7502,17:35,Attelé,5,PRIX DE CAUDEBEC,37 000,2 875,Course D,NaN,"Pour juments de 5 ans, n'ayant pas gagné 71.00...",True,True,16,7 - 11 - 9 - 2 - 4,https://www.letrot.com/stats/fiche-course/2021...,"<a href=""https://www.letrot.com/fr/replay-cour..."
9067,2021-03-23,2021032375026,4,ENGHIEN,7502,18:09,Attelé,6,PRIX DE CHENNEVIERES,18 000,2 875,Course R,NaN,"Course ""a reclamer""<br />Pour 4 et 5 ans, les ...",True,False,16,8 - 14 - 13 - 9 - 10,https://www.letrot.com/stats/fiche-course/2021...,"<a href=""https://www.letrot.com/fr/replay-cour..."


In [6]:
# courses_groupes = courses.courses[courses.courses["categorie"].isin(['Groupe I','Groupe II','Groupe III'])]

In [19]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler,StandardScaler

class DataError(ValueError): pass
class PMUError(Exception): pass

class Partants():
    def __init__(self, course, training=True):
        
        to_scale = ['nombreCourses',
                    'nombreVictoires',
                    'nombrePlaces',
                    'nombrePlacesSecond',
                    'nombrePlacesTroisieme',
                    'gainsParticipant_gainsCarriere',
                    'gainsParticipant_gainsVictoires',
                    'gainsParticipant_gainsPlace',
                    'gainsParticipant_gainsAnneeEnCours',
                    'gainsParticipant_gainsAnneePrecedente',
                    'nbDiscalifieMusic',
                    'nbVictoireMusic',
                    'nbPlaceMusic',
                    'prefered_dist',
                    'distToPreferedDist',
                    'meanReduction',
                    'medianReduction',
                    'maxReduction',
                    'minReduction',
                    'timeSinceRecord',
                    'tpsLastRace',
                    'nbArriveMusic',
                    'recordAbs',
                    'nbCourseCouple',
                    'nbVictoiresCouple',
                    'nb2emeCouple',
                    'nb3emeCouple',
                    'txReussiteCouple',
                    'nbCourseHippo',
                    'nbVictoiresHippo',
                    'nb2emeHippo',
                    'nb3emeHippo',
                    'txReussiteHippo',
                    "mean_reduc_driver",
                    "prefered_dist_driver",
                    "20_mean_reduc_driver", 
                    "50_mean_reduc_driver", 
                    "median_reduc_driver",
                    "std_reduc_driver",
                    "mean_reduc_trainer",
                    "prefered_dist_trainer",
                    "20_mean_reduc_trainer", 
                    "50_mean_reduc_trainer", 
                    "median_reduc_trainer",
                    "std_reduc_trainer",
                    "10_mean_reduc_horse",
                    "median_reduc_horse",
                    "std_reduc_horse",
                    "gains_gagnant_ratio_horse",
                    "gains_place_ratio_horse",
                    "gains_gagnant_ratio_driver",
                    "gains_place_ratio_driver",
                    "gains_gagnant_ratio_trainer",
                    "gains_place_ratio_trainer"]
        
        self.course = course
        self.courseId = course["id"]
        self.date = course["date"]
        self.idHippo = course["idHippo"]
        self.numCourse = course["numCourse"]
        self.numReunion = course["numReunion"]
        self.heure = course["heureCourse"]
        self.r_tab_partant = None
        self.r_tab_arrivee = None
        self.r_partant_pmu = None
        
        
        self.distance = int(course["distance"].replace(" ", ""))
        self.categorie = course["categorie"].split(" ")[1]
        
        self.training = training
        self.classement = None
        
        scaler = StandardScaler()
        
        
        
        try:
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            programme = loop.run_until_complete(self._info_tableau_partant())
        
        
#             self.info_partants = self._info_tableau_partant()
            df = pd.DataFrame(self.info_partants)
            df.loc[:, df.columns.isin(to_scale)] = scaler.fit_transform(df.loc[:, df.columns.isin(to_scale)].to_numpy())
            
            self.info_partants = df.to_dict('records')
        except:
            self.info_partants = None
        
        
    async def _request_tableau_partants(self, session):
        async with session.get(f"https://www.letrot.com/stats/fiche-course/{self.date}/{self.idHippo}/{self.numCourse}/partants/tableau", headers=headers) as response:
             r = await response.text()
        soup = bs(r, "html.parser")
        headers_table = soup.find("table", {"id": "result_table"}).find("thead").find("tr").find_all("th")
        table = soup.find("table", {"id": "result_table"}).find("tbody")
        rows = table.find_all("tr")
        self.r_tab_partant = rows, headers_table
    
    async def _request_tableau_arrive(self, session):
        async with session.get(f"https://www.letrot.com/stats/fiche-course/{self.date}/{self.idHippo}/{self.numCourse}/resultats/arrivee-definitive", headers=headers) as response:
             r = await response.text()
        soup = bs(r, "html.parser")
        headers_table = soup.find("table", {"id": "result_table"}).find("thead").find("tr").find_all("th")
        table = soup.find("table", {"id": "result_table"}).find("tbody")
        rows = table.find_all("tr")
        
        classement = {row.select("td")[1].text : row.select("td")[0].find("span", {"class": "bold"}).text for row in rows}       
        self.classement = classement
        self.r_tab_arrivee = rows,classement
    
    async def _request_partant_pmu(self, session):
        date_pmu = "".join(self.date.split("-")[::-1]) 
        async with session.get(f"https://online.turfinfo.api.pmu.fr/rest/client/65/programme/{date_pmu}/R{self.numReunion}/C{self.numCourse}/participants", headers=headers) as response:
             participants_pmu = await response.json()
        try:
            pmu_jsoned = participants_pmu["participants"]
            participants = pd.json_normalize(pmu_jsoned, sep="_").to_dict(orient="records")
            participants_with_id = [dict(item, **{"id": self.courseId, "numReunion": self.numReunion}) for item in participants]  
            self.r_partant_pmu = participants_with_id
        except:
            raise PMUError("Erreur API PMU")
            
        
        
    async def _info_tableau_partant(self):
        chevaux = []
        
        tasks = []
        async with aiohttp.ClientSession() as session:
            tasks.append(asyncio.ensure_future(self._request_tableau_partants(session)))
            tasks.append(asyncio.ensure_future(self._request_tableau_arrive(session)))
            tasks.append(asyncio.ensure_future(self._request_partant_pmu(session)))
            tasks.append(asyncio.ensure_future(self.get_info_couple(session)))
            tasks.append(asyncio.ensure_future(self.get_info_cheval_hippo(session)))
            res = await asyncio.gather(*tasks, return_exceptions=True)
            
        for r in res:
            if type(r) == PMUError:
                raise PMUError("dsfgkjh")
       
        try:
            tableau_partants, headers_table = self.r_tab_partant
            tableau_arrivee,classement = self.r_tab_arrivee
            tableau_pmu = self.r_partant_pmu
            
        except:
            return None
        try:
            info_chevaux_hippo = self.info_cheval_hippo
            info_couple = self.couple_info
        except:
            pass
    
        chevaux.extend(tableau_pmu)
        
        for i,row in enumerate(tableau_partants):
                num = row.select("td")[0].find("span", {"class": "bold"}).text
                col = row.select("td")
                cheval = {}
                cheval["num"] = num
                cheval["nom"] = col[1].text
                
                
                
                cheval["numCoursePMU"] = f"R{self.numReunion}C{self.numCourse}"

                if self.training:
                    if num == "NP":
                        cheval["classement"] = "NP"
                    else:
                        cheval["classement"] = classement[num]
                cheval["id"] = self.courseId
                cheval["date"] = self.date
                cheval["url"] = col[1].find("a").get("href")

                cheval["heureCourse"] = self.heure
                cheval["fer"] = int(col[3].text) if col[3].text else 0
                cheval["firstTimeFer"] = 1 if col[3].find("div", {"class", "fer-first-time"}) else 0
                cheval["sex"] = 0 if col[4].text == "M" else 1
                cheval["age"] = int(col[5].text)
                cheval["dist"] = int(col[6].text.replace(" ", "").replace("\n", ""))
                cheval["driver"] = col[7].find("a").get("href")
                cheval["trainer"] = col[8].find("a").get("href")

                if "Avis" in headers_table[9].text:
                    cheval["avisTrainer"] = int(col[9].get("data-order"))
                    avis = col.pop(9)
                    col.insert(-1, avis)
                else:
                    cheval["avisTrainer"] = 2

                cheval["music"] = list(filter(lambda x: "a" in x, col[9].text.replace("D", "0").replace("Ret", "0").replace("T", "0").split()))
                cheval["music"] = list(map(lambda x: x[0], cheval["music"]))

                cheval["music"] = list(filter(lambda x: x.isnumeric(), cheval["music"]))

                cheval["music"] = list(map(int, cheval["music"]))

                cheval["nbDiscalifieMusic"] = cheval["music"].count(0)
                cheval["nbVictoireMusic"] = cheval["music"].count(1)
                cheval["nbPlaceMusic"] = sum(map(lambda x : x <=3 and x > 0,cheval["music"]))
                
                
                if len(cheval["music"]) < 4:
                    raise DataError("not enough data")
                
                t_cheval = []
                async with aiohttp.ClientSession() as session:
                    t_cheval.append(asyncio.ensure_future(self.get_info_cheval(session,cheval["url"], self.date,cheval["driver"])))
                    t_cheval.append(asyncio.ensure_future(self.get_info_driver(session,cheval["driver"])))
                    t_cheval.append(asyncio.ensure_future(self.get_info_trainer(session,cheval["trainer"])))
                    t_cheval.append(asyncio.ensure_future(self.get_tracking(session,cheval["url"])))
                    res = await asyncio.gather(*t_cheval, return_exceptions=True)
                for r in res:
                    if type(r) == DataError:
                        raise DataError("dsfgkjh")
                for dic in res:
                    cheval.update(dic)

                try:
                    cheval.update(info_couple[i])
                    cheval.update(info_chevaux_hippo[i])
                except:
                    pass

                cheval["formeVictoire"] = 1 if cheval["nbVictoireMusic"]/len(cheval["music"]) > 0.33 else 0
                cheval["formePlace"] = 1 if cheval["nbPlaceMusic"]/len(cheval["music"]) > 0.33 else 0
                    
                cheval["nbArriveMusic"] = len(cheval["music"]) - cheval["music"].count(0)
                cheval["lastPerf"] = cheval["music"][0] if cheval["nbArriveMusic"] else 0

                arriveOnly = list(filter(None, cheval["music"]))
                if len(arriveOnly) > 0:
                    try:
                        cheval["meanPerf"] = np.mean(arriveOnly)
                        cheval["medianPerf"] = np.median(arriveOnly)
                        cheval["modePerf"] = st.mode(cheval["music"])
                    except:
                        cheval["meanPerf"] = 0
                        cheval["medianPerf"] = 0
                        cheval["modePerf"] = 0
                else:
                    cheval["meanPerf"] = 0
                    cheval["medianPerf"] = 0
                    cheval["modePerf"] = 0

                try:
                    cheval["recordAbs"] = list(map(int, col[10].text.replace(col[10].span.text, "").replace("\'", '"').split('"')))
                    cheval["recordAbs"] = cheval["recordAbs"][0] * 10 * 60 + cheval["recordAbs"][1] * 10 + cheval["recordAbs"][2]
                except:
                    cheval["recordAbs"] = None

                cheval["gain"] = int(col[11].find("div", class_="gains").text.replace(" ", "")[:-1])
                
                chevaux[i].update(cheval)
        self.info_partants = chevaux
    
    async def get_info_cheval(self,session, url, date, driver):
        async with session.get(url + "-paginate-2", headers=headers) as response:
             r = await response.json()
        date_debut = datetime.date.fromisoformat(date)
        jsoned = r["data"]

        info_dict = {}

        for c in jsoned:
            c["dateCourse"] = datetime.date.fromisoformat(c["dateCourseRaw"])
            c["categorie"] = bs(c["categorie"], "html.parser").find("span").text
            try:
                c["rang"] = min(int(bs(c["rang"], "html.parser").find("span").text.lstrip("0")), 10)
            except:
                c["rang"] = 10

            try:
            
                c["driver"] = bs(c["nomDriver"], "html.parser").find("a").get("href").split("/")[-3] == driver.split("/")[-3]
            except:
                c["driver"] = 0
            reduction = bs(c["reduction"], "html.parser").span.text
            reduction = reduction.replace("'", "").replace('\"', "")
            try:
                c["allocation"] = int(bs(c["allocation"], "html.parser").span.text.lstrip("0"))
            except:
                c["allocation"] = 0
            

            c["distance"] = int(c["distance"].replace(" ", "")) if c["distance"] != None else None

            reduction_min = int(str(reduction)[0])
            reduction_sec = int(str(reduction)[1:3])
            reduction_ssec = int(str(reduction)[3])

            c["reduction"] = reduction_min*60*10 + reduction_sec*10 + reduction_ssec

            # c["recordAbs"] = list(map(int, reduction.text.replace(reduction.span.text, "").replace("\'", '"').split('"')))
            # c["recordAbs"] = c["recordAbs"][0] * 10 * 60 + c["recordAbs"][1] * 10 + c["recordAbs"][2]

        filtered = list(filter(lambda x: x["dateCourse"] < date_debut and x["specialite"] == "A", jsoned))
        
        wins_cheval = pd.DataFrame(filtered)
#         print(wins_cheval["rang"])
        if len(wins_cheval[wins_cheval["rang"] == 1]) >= 1:
            gains_gagnant = wins_cheval[wins_cheval["rang"] == 1]["allocation"].sum() / len(wins_cheval[wins_cheval["rang"] == 1])
        else:
            gains_gagnant = 0
        if len(wins_cheval[wins_cheval["rang"] <= 3]) >= 1:
            gains_place = wins_cheval[wins_cheval["rang"] <= 3]["allocation"].sum() / len(wins_cheval[wins_cheval["rang"] <= 3])
        else:
            gains_place = 0
        info_dict["gains_gagnant_ratio_horse"] = gains_gagnant
        info_dict["gains_place_ratio_horse"] = gains_place
        
        
        if len(filtered) == 0:
            raise DataError("Not enough Data")
        
        perc_jockey = sum([x["driver"] for x in filtered])/len(filtered)
        info_dict["jockeyHabitude"] = 1 if perc_jockey > 0.5 else 0
        
        last_30_days = list(filter(lambda x: x["dateCourse"] > date_debut - datetime.timedelta(days=30) and x["specialite"] == "A", jsoned))
        
        weights = [ max(x["allocation"], 1) for x in filtered if x["distance"] != None  ]
        if sum(weights) == 0:
            raise DataError("Not enough Data")
#         print(weights)
        prefered_dist = int(np.average([ x["distance"] for x in filtered if x["distance"] != None ], weights=weights))
        
        info_dict["prefered_dist"] = prefered_dist
        info_dict["distToPreferedDist"] = abs(self.distance - prefered_dist)
        
        info_dict["changementCategorie"] = 1 if filtered[0]["categorie"] != self.categorie else 0
        
        if len(last_30_days) > 0:
            dist_30_days = np.array([x["distance"] for x in last_30_days])
            info_dict["newDist"] = 1 if np.max(dist_30_days - self.distance) > 200 else 0
        else:
            info_dict["newDist"] = 1
            
        
        filtered_tps = list(filter(lambda x: x["reduction"] < 1200, filtered))
        
        dps_race = [(date_debut - x["dateCourse"]).days for x in filtered_tps]
        tps = [x["reduction"] for x in filtered_tps]
        tps_series = pd.Series(tps[::-1]).dropna() 
        
        if len(tps) > 0:
            info_dict["meanReduction"] = np.mean(tps)
            info_dict["medianReduction"] = np.median(tps)
            info_dict["maxReduction"] = max(tps)
            info_dict["minReduction"] = min(tps)
            
            info_dict["10_mean_reduc_horse"] = tps_series.ewm(len(tps_series)-1).mean().iloc[-1]
            info_dict["median_reduc_horse"] = tps_series.median()
            info_dict["std_reduc_horse"] = tps_series.std()
            
            lin_reg = LinearRegression().fit(np.array(dps_race).reshape(-1, 1), np.array(tps).reshape(-1, 1))
            info_dict["progressTps"] = np.exp(lin_reg.coef_[0][0])
            
        else:
            info_dict["meanReduction"] = 0
            info_dict["medianReduction"] = 0
            info_dict["maxReduction"] = 0
            info_dict["minReduction"] = 0
            info_dict["progressTps"] = 0

        info_dict["timeSinceRecord"] = next(((date_debut - item["dateCourse"]).days for item in filtered_tps if item["reduction"] == info_dict["minReduction"]), 365)

        info_dict["tpsLastRace"] = (date_debut - filtered[0]["dateCourse"]).days

        info_dict["last_race_dist"] = filtered[0]["distance"]
        info_dict["rentree"] = 1 if info_dict["tpsLastRace"] > 30 else 0

        return info_dict

    async def get_info_driver(self,session, url):

        date = datetime.date.fromisoformat(self.date)
        
        d = datetime.timedelta(days=1)
        d2 = datetime.timedelta(days=365)

        date_arrive = (date - d).strftime("%d-%m-%Y").replace("-", "%2F")
        date_depart = (date  - d2).strftime("%d-%m-%Y").replace("-", "%2F")
        driver_id = url.split("/")[-3]
        n_url = f"https://www.letrot.com/stats/fiche-homme/analysesperformances-paginate?lenght=100&firstTime=true&hom_type=jockey&hom_num_encode={driver_id}&ferrure=all&hippodrome=all&corde=all&sol=all&discipline=A&depart=all&datepicker_du={date_depart}&datepicker_au={date_arrive}"
        
        

        
        async with session.get(n_url.replace("paginate", "synthese"), headers=headers) as response:
            synthese = await response.json()
        synthese = {str(key) + "_driver": val for key,val in synthese.items()}
        
        async with session.get(n_url, headers=headers) as response:
             r = await response.json()
        jsoned = r["data"]
        info_dict = synthese
        
        for d in jsoned:
            d["dateCourse"] = datetime.date.fromisoformat(bs(d["date_course"], "html.parser").span.text)
            try:
                d["rang"] = min(int(bs(d["rang"], "html.parser").find("span").text.lstrip("0")), 10)
            except:
                d["rang"] = 10

            reduction = d["reduction"].replace("'", "").replace('\"', "")
            
            if reduction != "":
                reduction_min = int(str(reduction)[0])
                reduction_sec = int(str(reduction)[1:3])
                reduction_ssec = int(str(reduction)[3])
                d["reduction"] = reduction_min*60*10 + reduction_sec*10 + reduction_ssec
            else:
                d["reduction"] = np.nan
            
            d["distance"] = int(d["distance"].replace(" ", "")) if d["distance"] != None else None
            try:
                d["allocation"] = int(bs(d["allocation"], "html.parser").span.text.lstrip("0"))
            except:
                d["allocation"] = 0

        wins_cheval = pd.DataFrame(jsoned)
        if len(wins_cheval[wins_cheval["rang"] == 1]) >= 1:
            gains_gagnant = wins_cheval[wins_cheval["rang"] == 1]["allocation"].sum() / len(wins_cheval[wins_cheval["rang"] == 1])
        else:
            gains_gagnant = 0
        if len(wins_cheval[wins_cheval["rang"] <= 3]) >= 1:
            gains_place = wins_cheval[wins_cheval["rang"] <= 3]["allocation"].sum() / len(wins_cheval[wins_cheval["rang"] <= 3])
        else:
            gains_place = 0
        info_dict["gains_gagnant_ratio_driver"] = gains_gagnant
        info_dict["gains_place_ratio_driver"] = gains_place
        
        weights = [ max(x["allocation"], 1) for x in jsoned]
        if sum(weights) == 0:
            raise DataError("Not enough Data")

        prefered_dist = int(np.average([ x["distance"] for x in jsoned if x["distance"] != None ], weights=weights))
        
        reduc_series = pd.Series([d["reduction"] for d in jsoned if d["reduction"] < 1200][::-1]).dropna()
        
        info_dict["20_mean_reduc_driver"] = reduc_series.ewm(20).mean().iloc[-1]
        info_dict["50_mean_reduc_driver"] = reduc_series.ewm(50).mean().iloc[-1]
        info_dict["median_reduc_driver" ]= reduc_series.median()
        info_dict["std_reduc_driver"] = reduc_series.std()
        
        info_dict["prefered_dist_driver"] = prefered_dist
        
        return info_dict
    
    async def get_info_trainer(self,session, url):

        date = datetime.date.fromisoformat(self.date)

        d = datetime.timedelta(days=1)
        d2 = datetime.timedelta(days=365)

        date_arrive = (date - d).strftime("%d-%m-%Y").replace("-", "%2F")
        date_depart = (date  - d2).strftime("%d-%m-%Y").replace("-", "%2F")
        trainer_id = url.split("/")[-3]
        n_url = f"https://www.letrot.com/stats/fiche-homme/analysesperformances-paginate?lenght=100&firstTime=true&hom_type=entraineur&hom_num_encode={trainer_id}&ferrure=all&hippodrome=all&corde=all&sol=all&discipline=A&depart=all&datepicker_du={date_depart}&datepicker_au={date_arrive}"

        async with session.get(n_url.replace("paginate", "synthese"), headers=headers) as response:
            synthese = await response.json()
        synthese = {str(key) + "_trainer": val for key,val in synthese.items()}

        async with session.get(n_url, headers=headers) as response:
             r = await response.json()
        jsoned = r["data"]
        info_dict = synthese

        for d in jsoned:
            d["dateCourse"] = datetime.date.fromisoformat(bs(d["date_course"], "html.parser").span.text)
            try:
                d["rang"] = min(int(bs(d["rang"], "html.parser").find("span").text.lstrip("0")), 10)
            except:
                d["rang"] = 10

            reduction = d["reduction"].replace("'", "").replace('\"', "")

            if reduction != "":
                reduction_min = int(str(reduction)[0])
                reduction_sec = int(str(reduction)[1:3])
                reduction_ssec = int(str(reduction)[3])
                d["reduction"] = reduction_min*60*10 + reduction_sec*10 + reduction_ssec
            else:
                d["reduction"] = np.nan

            d["distance"] = int(d["distance"].replace(" ", "")) if d["distance"] != None else None
            try:
                d["allocation"] = int(bs(d["allocation"], "html.parser").span.text.lstrip("0"))
            except:
                d["allocation"] = 0

        wins_cheval = pd.DataFrame(jsoned)
        if len(wins_cheval[wins_cheval["rang"] == 1]) >= 1:
            gains_gagnant = wins_cheval[wins_cheval["rang"] == 1]["allocation"].sum() / len(wins_cheval[wins_cheval["rang"] == 1])
        else:
            gains_gagnant = 0
        if len(wins_cheval[wins_cheval["rang"] <= 3]) >= 1:
            gains_place = wins_cheval[wins_cheval["rang"] <= 3]["allocation"].sum() / len(wins_cheval[wins_cheval["rang"] <= 3])
        else:
            gains_place = 0
        info_dict["gains_gagnant_ratio_trainer"] = gains_gagnant
        info_dict["gains_place_ratio_trainer"] = gains_place

        weights = [ max(x["allocation"], 1) for x in jsoned]
        if sum(weights) == 0:
            raise DataError("Not enough Data")

        prefered_dist = int(np.average([ x["distance"] for x in jsoned if x["distance"] != None ], weights=weights))

        reduc_series = pd.Series([d["reduction"] for d in jsoned if d["reduction"] < 1200][::-1]).dropna()

        info_dict["20_mean_reduc_trainer"] = reduc_series.ewm(20).mean().iloc[-1]
        info_dict["50_mean_reduc_trainer"] = reduc_series.ewm(50).mean().iloc[-1]
        info_dict["median_reduc_trainer" ]= reduc_series.median()
        info_dict["std_reduc_trainer"] = reduc_series.std()

        info_dict["prefered_dist_trainer"] = prefered_dist

        return info_dict
    
    async def get_info_couple(self, session):
        couple_info = []

        date = datetime.date.fromisoformat(self.date)
        
        d = datetime.timedelta(days=1)
        d2 = datetime.timedelta(days=365)

        date_arrive = (date - d).strftime("%d-%m-%Y").replace("-", "%2F")
        date_depart = (date  - d2).strftime("%d-%m-%Y").replace("-", "%2F")
        
        url = f"https://www.letrot.com/stats/fiche-course/{self.date}/{self.idHippo}/{self.numCourse}/partants/couples/paginate?datepicker_du={date_depart}&datepicker_au={date_arrive}"
                
        async with session.get(url, headers=headers) as response:
             r = await response.json()
        data = r["data"]

        data_sorted = sorted(data, key=lambda x: x["numero"])
        for couple in data_sorted:
            cheval = {}

            cheval["nbCourseCouple"] = int(bs(couple["nbre_courses"], "html.parser").find("div").text)
            cheval["nbVictoiresCouple"] = int(bs(couple["nbre_victoires"], "html.parser").find("div").text)
            cheval["nb2emeCouple"] = int(bs(couple["nbre_2eme"], "html.parser").find("div").text)
            cheval["nb3emeCouple"] = int(bs(couple["nbre_3eme"], "html.parser").find("div").text)
            cheval["txReussiteCouple"] = int(couple["taux_reussite_sort"])/100
            try:
                cheval["txVictoireCouple"] = cheval["nbVictoiresCouple"] / cheval["nbCourseCouple"]
            except:
                cheval["txVictoireCouple"] = 0.0
            cheval["nonPartant"] = couple["nonPartant"]
            cheval["moreFirstThanThirdCouple"] = 1 if cheval["nbVictoiresCouple"] > cheval["nb3emeCouple"] + cheval["nb2emeCouple"] else 0
            couple_info.append(cheval)
        self.couple_info = couple_info
    
    async def get_info_cheval_hippo(self, session):
        couple_info = []

        date = datetime.date.fromisoformat(self.date)
        
        d = datetime.timedelta(days=1)
        d2 = datetime.timedelta(days=365)

        date_arrive = (date - d).strftime("%d-%m-%Y").replace("-", "%2F")
        date_depart = (date  - d2).strftime("%d-%m-%Y").replace("-", "%2F")
        url = f"https://www.letrot.com/stats/fiche-course/{self.date}/{self.idHippo}/{self.numCourse}/partants/chevaux/paginate?numHippodrome={self.idHippo}&piste=all&datepicker_du={date_depart}&datepicker_au={date_arrive}"
        
        async with session.get(url, headers=headers) as response:
             r = await response.json()
        data = r["data"]

        data_sorted = sorted(data, key=lambda x: x["numero"])
        for couple in data_sorted:
            cheval = {}

            cheval["nbCourseHippo"] = int(bs(couple["nbre_courses"], "html.parser").find("div").text)
            cheval["nbVictoiresHippo"] = int(bs(couple["nbre_victoires"], "html.parser").find("div").text)
            cheval["nb2emeHippo"] = int(bs(couple["nbre_2eme"], "html.parser").find("div").text)
            cheval["nb3emeHippo"] = int(bs(couple["nbre_3eme"], "html.parser").find("div").text)
            try:
                cheval["txVictoireHippo"] = cheval["nbVictoiresHippo"] / cheval["nbCourseHippo"]
            except:
                cheval["txVictoireHippo"] = 0.0
            try:
                cheval["txReussiteHippo"] =int(couple["taux_reussite_sort"])/100
            except:
                cheval["txReussiteHippo"] = 0.0
                                              
            cheval["perfHippo"] = 1 if cheval["txReussiteHippo"] > 0.5 and cheval["nbCourseHippo"] > 5 else 0
            couple_info.append(cheval)
        self.info_cheval_hippo = couple_info
    
    async def get_tracking(self,session, url):
        async with session.get(url.replace("dernieres-performances", "tracking"), headers=headers) as response:
             r = await response.text()
        soup = bs(r, "html.parser")
        headers_table = soup.find("table", {"id": "result_table"}).find("thead").find("tr").find_all("th")
        table = soup.find("table", {"id": "result_table"}).find("tbody")
        rows = table.find_all("tr")
        
        info_tracking = {}
        
        distance_au_premier_arrivee = []
        accélération_500m = []
        gain_classement_500m = []
        for row in rows:
            dist_prem = int(row.find_all("td")[2].span.text)
            if dist_prem < 9999:
                distance_au_premier_arrivee.append(dist_prem)
            
            pre_fin = int(row.find_all("td")[17].span.text)
            fin = int(row.find_all("td")[18].span.text)
            if pre_fin < 2000 and fin < 2000:                                            
                accélération_500m.append(pre_fin - fin)
                

            try:
                class_500m = int(row.find_all("td")[16].span.text)
                class_final = int(row.find_all("td")[1].find("span", {"class": "bold"}).text)
                if class_500m -  class_final < 10:
                    gain_classement_500m.append(class_500m -  class_final)
            except:
                gain_classement_500m.append(0)
            
        info_tracking["mean_dist_arrivee"] = np.mean(distance_au_premier_arrivee) if len(distance_au_premier_arrivee) > 0 else np.nan
        info_tracking["acceleration_500m"] = np.mean(accélération_500m) if len(accélération_500m) > 0 else np.nan
        info_tracking["gain_classement_fin"] = np.mean(gain_classement_500m) if len(gain_classement_500m) > 0 else np.nan
        return info_tracking

In [ ]:
info = []
t = time.time()
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    res = executor.map(Partants, gen_rows(c.iloc[700:702]))
    for i in res:
        if isinstance(i.info_partants, list):
            info.extend(i.info_partants)
#             pd.DataFrame(i.info_partants).to_csv("data/new_data2122.csv",mode="a", header=not os.path.isfile("data/new_data2122.csv") ,index=False)

tps = time.time() - t
print(f"Fini en {int(tps//60)}min {int(tps%60)}s")

In [ ]:
# info_2 = []
t = time.time()
counter = 0

ids = pd.read_csv("data/final_data_2122.csv").id.unique()
c_filtered = c.loc[~c.id.isin(ids)]

for row in gen_rows(c_filtered.iloc[20:]):
    res = Partants(row)
    if isinstance(res.info_partants, list):
#         info_2.extend(res.info_partants)
        pd.DataFrame(res.info_partants, columns=cols).to_csv("data/final_data_2122.csv",mode="a", header=not os.path.isfile("data/final_data_2122.csv") ,index=False)
        counter+=1
        if counter%20 == 0:
            print(counter, "courses enregistree")
tps = time.time() - t
print(f"Fini en {int(tps//60)}min {int(tps%60)}s")

C:\Users\raves\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (151) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
<ipython-input-19-7763b3e9f918>:451: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  reduc_series = pd.Series([d["reduction"] for d in jsoned if d["reduction"] < 1200][::-1]).dropna()
<ipython-input-19-7763b3e9f918>:451: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  reduc_series = pd.Series([d["reduction"] for d in jsoned if d["reduction"] < 1200][::-1]).dropna()
<ipython-input-19-7763b3e9f918>:524: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future v

20 courses enregistree


<ipython-input-19-7763b3e9f918>:451: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  reduc_series = pd.Series([d["reduction"] for d in jsoned if d["reduction"] < 1200][::-1]).dropna()
<ipython-input-19-7763b3e9f918>:451: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  reduc_series = pd.Series([d["reduction"] for d in jsoned if d["reduction"] < 1200][::-1]).dropna()
<ipython-input-19-7763b3e9f918>:524: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  reduc_series = pd.Series([d["reduction"] for d in jsoned if d["reduction"] < 1200][::-1]).dropna()
<ipython-input-19-7763b3e9f918>:451: DeprecationWarning: The default dtype for empty Ser

<ipython-input-19-7763b3e9f918>:524: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  reduc_series = pd.Series([d["reduction"] for d in jsoned if d["reduction"] < 1200][::-1]).dropna()
<ipython-input-19-7763b3e9f918>:451: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  reduc_series = pd.Series([d["reduction"] for d in jsoned if d["reduction"] < 1200][::-1]).dropna()
<ipython-input-19-7763b3e9f918>:451: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  reduc_series = pd.Series([d["reduction"] for d in jsoned if d["reduction"] < 1200][::-1]).dropna()
<ipython-input-19-7763b3e9f918>:451: DeprecationWarning: The default dtype for empty Ser

<ipython-input-19-7763b3e9f918>:451: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  reduc_series = pd.Series([d["reduction"] for d in jsoned if d["reduction"] < 1200][::-1]).dropna()
<ipython-input-19-7763b3e9f918>:451: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  reduc_series = pd.Series([d["reduction"] for d in jsoned if d["reduction"] < 1200][::-1]).dropna()
<ipython-input-19-7763b3e9f918>:451: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  reduc_series = pd.Series([d["reduction"] for d in jsoned if d["reduction"] < 1200][::-1]).dropna()
<ipython-input-19-7763b3e9f918>:451: DeprecationWarning: The default dtype for empty Ser

...

600 courses enregistree


<ipython-input-19-7763b3e9f918>:451: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  reduc_series = pd.Series([d["reduction"] for d in jsoned if d["reduction"] < 1200][::-1]).dropna()
<ipython-input-19-7763b3e9f918>:524: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  reduc_series = pd.Series([d["reduction"] for d in jsoned if d["reduction"] < 1200][::-1]).dropna()
<ipython-input-19-7763b3e9f918>:451: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  reduc_series = pd.Series([d["reduction"] for d in jsoned if d["reduction"] < 1200][::-1]).dropna()
<ipython-input-19-7763b3e9f918>:451: DeprecationWarning: The default dtype for empty Ser

620 courses enregistree


<ipython-input-19-7763b3e9f918>:524: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  reduc_series = pd.Series([d["reduction"] for d in jsoned if d["reduction"] < 1200][::-1]).dropna()
<ipython-input-19-7763b3e9f918>:451: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  reduc_series = pd.Series([d["reduction"] for d in jsoned if d["reduction"] < 1200][::-1]).dropna()
<ipython-input-19-7763b3e9f918>:451: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  reduc_series = pd.Series([d["reduction"] for d in jsoned if d["reduction"] < 1200][::-1]).dropna()


640 courses enregistree


RUN SANS ASYNCIO: 37.02154350280762 secondes

In [16]:
df = pd.DataFrame(info_2)
df

,nom,numPmu,age,sexe,race,statut,oeilleres,proprietaire,entraineur,deferre,...,nbArriveMusic,lastPerf,meanPerf,medianPerf,modePerf,recordAbs,gain,placeCorde,ecurie,poidsConditionMonte
0,INTUITION SOYER,1,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,J.L. TIRAPU,A. RANDON,DEFERRE_ANTERIEURS_POSTERIEURS,...,0.262613,8,5.000000,7.0,8,1.636397,15430,NaN,NaN,NaN
1,IDOLE DARLING,2,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,Mlle C. BEILEARD,M. MOTTIER,DEFERRE_ANTERIEURS_POSTERIEURS,...,-0.787839,5,3.400000,3.0,3,0.796730,17710,NaN,NaN,NaN
2,ISIS PASSION,3,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,Ecurie de ROUGEMONT,L. PESCHET,DEFERRE_ANTERIEURS_POSTERIEURS,...,-1.313064,7,4.250000,4.5,0,0.720397,18750,NaN,NaN,NaN
3,IPSWICH DE CAHOT,4,4,HONGRES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,S. PRIOUL,S. PRIOUL,DEFERRE_POSTERIEURS,...,-0.787839,1,3.600000,3.0,0,-0.042938,19060,NaN,NaN,NaN
4,INDIE BAKALAIKA,5,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,Quadra PALITO,A. RIPOLL RIGO,DEFERRE_ANTERIEURS_POSTERIEURS,...,0.787839,1,3.750000,3.5,1,0.567730,19450,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,GOLDEN VISAIS,9,6,HONGRES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,B. HERBILLON,C. BUHIGNE,NaN,...,1.168462,1,3.666667,3.0,3,-0.194330,170120,NaN,NaN,NaN
114,FINGER DES BORDES,10,7,HONGRES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,D. DORE,D. DORE,NaN,...,-0.212448,0,2.375000,2.0,0,-0.849295,173450,NaN,NaN,NaN
115,FEELING BOY,11,7,HONGRES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,Y. DREUX,Y. DREUX,DEFERRE_POSTERIEURS,...,0.478007,0,4.400000,4.5,2,-0.568596,175895,NaN,NaN,NaN
116,GALA DE CRENNES,12,6,MALES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,Ecurie Haras de CRENNES,E. SZIRMAY,PROTEGE_ANTERIEURS,...,0.478007,2,5.400000,6.5,7,-0.100764,186820,NaN,NaN,NaN


In [24]:
cols = list(pd.read_csv("data/new_data2122.csv"))
len(cols)

C:\Users\raves\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (150) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


171

In [115]:
test = [{"A": 5, "B":1, "C":7},{"A": 2, "B":2}]
pd.DataFrame(test, columns=["B","A","C"])

,B,A,C
0,1,5,7.0
1,2,2,NaN


In [7]:
len(c.drop_duplicates())

9069

In [6]:
df = pd.read_csv("data/final_data_2122.csv", index_col=0)
df.tail()

C:\Users\raves\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (151) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,nom,numPmu,age,sexe,race,statut,oeilleres,proprietaire,entraineur,deferre,...,nbArriveMusic,lastPerf,meanPerf,medianPerf,modePerf,recordAbs,gain,placeCorde,ecurie,poidsConditionMonte
15399,JOLIE NANA,11,3,FEMELLES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,L. SWITA,L. SWITA,NaN,...,0.919866,9,5.125,5.0,0,-0.039113,25380,11.0,NaN,NaN
15437,FAST DES BROUSSES,7,7,HONGRES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,Ecurie EN AVANT GOUDIG,A. GARANDEAU,NaN,...,0.370625,0,4.000,4.0,2,-0.481321,135400,NaN,NaN,NaN
15438,GALBA RINGEAT,8,6,HONGRES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,Ecurie NIXONN,R. DERIEUX,DEFERRE_ANTERIEURS_POSTERIEURS,...,-0.296500,6,3.000,1.0,1,-0.164354,145009,NaN,NaN,NaN
15455,DE LA CHENEVIERE,16,9,FEMELLES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,N. BARBONI,L. GABORIT,PROTEGE_ANTERIEURS,...,1.378778,8,6.600,8.0,9,-1.165126,242900,NaN,NaN,NaN
15517,DREAM GOLD,14,9,HONGRES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,A. RODRIGUES PRETO,S. ROUSSEAU,NaN,...,-0.747590,0,7.500,7.5,0,-1.289317,210480,NaN,NaN,NaN


In [14]:
df2 = pd.read_csv("data/copie.csv", header=None, names=list(df))
df2.tail()

C:\Users\raves\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (30) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,nom,numPmu,age,sexe,race,statut,oeilleres,proprietaire,entraineur,deferre,...,nbArriveMusic,lastPerf,meanPerf,medianPerf,modePerf,recordAbs,gain,placeCorde,ecurie,poidsConditionMonte
8293,COPAIN MALOUIN,9,10,HONGRES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,K. DEVIENNE,J.M. ROUBAUD,DEFERRE_ANTERIEURS_POSTERIEURS,...,-0.364390,1,4.875000,5.5,0,-0.417299,171340,NaN,NaN,NaN
8294,MASSIVE ATTACK,10,9,MALES,TROTTEUR ETRANGER,PARTANT,SANS_OEILLERES,Mme G. CERSOSIMO,X. THIELENS,DEFERRE_ANTERIEURS_POSTERIEURS,...,0.227744,8,4.777778,6.0,1,-0.669619,176968,NaN,NaN,NaN
8295,ESPOIR DE BELLOUET,11,8,HONGRES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,Ecurie du VIEUX CHENE,X. THIELENS,NaN,...,-0.956524,9,4.571429,4.0,0,0.213502,179530,NaN,NaN,NaN
8296,DENZO MONTAVAL,12,9,HONGRES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,Mme B. CHAUDEMANCHE,P. TERRY,DEFERRE_ANTERIEURS_POSTERIEURS,...,0.227744,2,4.333333,5.0,1,-2.688180,184980,NaN,NaN,NaN
8297,FLICKA JULRY,13,7,FEMELLES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,Ecurie JULRY,J.P. GAUVIN,DEFERRE_ANTERIEURS_POSTERIEURS,...,0.819878,1,2.800000,1.5,1,-0.417299,188190,NaN,NaN,NaN


In [21]:
df[]

0        DA
1         0
2         8
3        DA
4         6
         ..
15399     4
15437     8
15438     7
15455     6
15517     0
Name: classement, Length: 12056, dtype: object

In [19]:
len(df2.groupby("id").filter(lambda x: 1 in x["classement"]))

16

In [13]:
len(pd.read_csv("data/final_data_2122.csv"))

C:\Users\raves\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (151) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


12056

In [108]:
# df.to_csv("data/newdata2122.csv")

In [38]:
df[["mean_dist_arrivee", "acceleration_500m","gain_classement_fin"]]

,mean_dist_arrivee,acceleration_500m,gain_classement_fin
0,42.000000,2.750000,-0.750000
1,30.500000,7.000000,1.000000
2,26.750000,0.250000,2.250000
3,8.000000,8.333333,0.333333
4,38.500000,18.500000,-1.666667
...,...,...,...
8739,18.285714,8.857143,1.142857
8740,15.000000,-7.800000,-1.111111
8741,17.000000,3.000000,1.222222
8742,18.125000,9.125000,1.300000


In [51]:
df

,nom,numPmu,age,sexe,race,statut,oeilleres,proprietaire,entraineur,deferre,...,nbArriveMusic,lastPerf,meanPerf,medianPerf,modePerf,recordAbs,gain,placeCorde,ecurie,poidsConditionMonte
0,ALWAYS EK,1,6,MALES,TROTTEUR ETRANGER,PARTANT,SANS_OEILLERES,Scuderia EFFEBI (ITY),F. SOULOY,DEFERRE_ANTERIEURS_POSTERIEURS,...,0.125000,0,2.750000,1.5,0,0.100000,322094,NaN,NaN,NaN
1,CHARLY DE L'AUNAY,2,10,HONGRES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,Christophe KEERHEM,J. GUELPA,DEFERRE_ANTERIEURS_POSTERIEURS,...,0.750000,8,4.111111,3.0,3,1.000000,359150,NaN,NaN,NaN
2,MARCELLO WIBB,3,7,HONGRES,TROTTEUR ETRANGER,PARTANT,SANS_OEILLERES,Ecurie BLACK AND WHITE (BEL),V. MARTENS,DEFERRE_ANTERIEURS_POSTERIEURS,...,0.375000,3,5.000000,5.0,3,0.250000,399909,NaN,NaN,NaN
3,FINE COLLINE,4,7,FEMELLES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,Ecurie LD-M. ABRIVARD,M. ABRIVARD,DEFERRE_ANTERIEURS,...,0.500000,0,2.285714,1.0,0,0.000000,412960,NaN,NaN,NaN
4,ELITE DE JIEL,5,8,FEMELLES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,Ecurie LUCK,J.L. DERSOIR,PROTEGE_ANTERIEURS_DEFERRRE_POSTERIEURS,...,0.625000,9,4.125000,3.0,0,0.350000,413440,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,HELLO JADE RUSH,7,5,FEMELLES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,Ecurie D L,M. DONIO,DEFERRE_POSTERIEURS,...,0.833333,4,3.444444,4.0,1,0.973684,81330,NaN,NaN,NaN
83,HERCULE MADRIK,8,5,HONGRES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,R. BAHIER,J.M. BAUDOUIN,DEFERRE_ANTERIEURS_POSTERIEURS,...,0.666667,0,3.750000,3.0,0,0.552632,86850,NaN,NaN,NaN
84,HOUSTON DISA,9,5,HONGRES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,Y. GAUTIER,Y. GAUTIER,DEFERRE_ANTERIEURS,...,0.666667,1,2.750000,2.0,2,0.605263,110980,NaN,NaN,NaN
85,HELIOS DES ARCS,10,5,HONGRES,TROTTEUR FRANCAIS,PARTANT,SANS_OEILLERES,Mme M.J. MARQUET,V. GOUIN,PROTEGE_ANTERIEURS_DEFERRRE_POSTERIEURS,...,0.833333,4,3.666667,3.0,4,0.710526,130820,NaN,NaN,NaN


In [4]:
r = requests.get("https://www.letrot.com/stats/fiche-cheval/fee-de-ranchval/ZWdbYwUFBAcM/courses/tracking", headers=headers)
soup = bs(r.text, "html.parser")

In [5]:
headers_table = soup.find("table", {"id": "result_table"}).find("thead").find("tr").find_all("th")
table = soup.find("table", {"id": "result_table"}).find("tbody")
rows = table.find_all("tr")

In [70]:
courses.courses.head()

,date,id,numReunion,hippodrome,idHippo,heureCourse,discipline,numCourse,prix,allocation,distance,categorie,typePiste,conditionsEngagement,hasTracking,hasVideoHeat,statut,classement,linkPrix,replay
0,2022-08-27,2022082775001,4,VINCENNES,7500,13:40,Attelé,1,PRIX DE SAINT-MANDE,12 000,2 100,Course F,,Amateurs<br />Départ à l'autostart<br />Pour 6...,True,False,16,7 - 9 - 4 - 3 - 2,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
1,2022-08-27,2022082775002,4,VINCENNES,7500,14:15,Attelé,2,PRIX DE MULHOUSE,39 000,2 700,Course E,,"Pour chevaux entiers et hongres de 5 ans, n'ay...",True,False,16,6 - 15 - 11 - 5 - 12,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
2,2022-08-27,2022082775003,4,VINCENNES,7500,14:50,Attelé,3,PRIX DE DOLE,39 000,2 100,Course D,,Départ à l'autostart<br />Pour pouliches de 4 ...,True,False,16,4 - 2 - 1 - 11 - 9,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
3,2022-08-27,2022082775005,4,VINCENNES,7500,16:18,Attelé,5,PRIX DE LURY SUR ARNON,51 000,2 100,Course B,,Course Européenne<br />Départ à l'autostart<br...,True,False,16,1 - 6 - 4 - 9 - 13,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
4,2022-08-27,2022082775006,4,VINCENNES,7500,16:53,Attelé,6,YEARLING CUP 2022 - ARQANA TROT,150 000,2 700,Groupe III,,Pour 3 ans.<br />Sont seuls admis à participer...,True,True,16,6 - 11 - 9 - 10 - 3,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."


In [17]:
def get_rapport_course(i):
    date = datetime.date.fromisoformat(i["date"])
    
    date_pmu = date.strftime("%d%m%Y")
    r = requests.get(f"https://online.turfinfo.api.pmu.fr/rest/client/61/programme/{date_pmu}/R{i['numReunion']}/C{i['numCourse']}/rapports-definitifs?combinaisonEnTableau=true", headers=headers)
    dict_bien = {}
    try:
        rjson = r.json()
    except:
        return None
    for j in rjson:
        if i["id"] in dict_bien:
            dict_bien[i["id"]].update({j["typePari"]: j})
        else:
            dict_bien[i["id"]] = {j["typePari"]: j}
    return dict_bien

In [18]:
all_rapport_dict = {}

with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    res = executor.map(get_rapport_course, gen_rows(courses.courses))
    for i in res:
        if isinstance(i, dict):
            all_rapport_dict.update(i)

In [13]:
get_rapport_course(courses.courses.iloc[0])["2022082775001"]["E_SIMPLE_GAGNANT"]["rapports"]

KeyError: '2022082775001'

In [19]:
len(all_rapport_dict)

3604

In [20]:
with open("data/rapport_nointer.json", "w") as f:
    json.dump(all_rapport_dict, f, indent=4)